In [1]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ce760e406561f4d7462713b6be691c164ad6da0a5ea2ab9516ad7d56e3cbc305
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [2]:
# import all libraries
from google.colab import files
import pandas as pd
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volatility import AverageTrueRange
from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice
import math
import pandas as pd
import numpy as np
import yfinance as yf

In [3]:
uploaded = files.upload()

Saving df2.csv to df2.csv


In [4]:
df = pd.read_csv('df2.csv')

In [5]:
df.head()

,index,list
0,400,"['DFS', 'MKTX', 'SPGI', 'AXON', 'IQV', 'WRK', ..."
1,401,"['HOLX', 'TFX', 'HRL', 'APD', 'META', 'EXC', '..."
2,402,"['GOOGL', 'CRM', 'AME', 'FLT', 'HOLX', 'ADM', ..."
3,403,"['CMA', 'D', 'IFF', 'EVRG', 'DAL', 'CBOE', 'IN..."
4,404,"['DOV', 'LMT', 'BWA', 'EXPD', 'CPT', 'CTAS', '..."


In [6]:
df.iloc[0].list

"['DFS', 'MKTX', 'SPGI', 'AXON', 'IQV', 'WRK', 'QCOM', 'GPC', 'JCI', 'EXC']"

In [7]:
class Common_Class():
  """
  A common class to calculate the results for all strategies.
  Key Terms;
      - A bar is a unit of data at a given time, depends on the interval you choose, it provides you OHLCV and time info
  Methods in class:
  1. prepare_data()
      - Loads the close prices of all stocks and calculates the return of each stock
  2. close_graph(stock)
      - A line plot showcasing the close prices of all stocks over the stated period (start - end)
  3. return_date_price(stock)
      - Returns the date and price of the stock at that given date
  4. realised_balance(bar)
      - Returns the realised capital in your account in a given bar
  5. unrealised_balance(bar)
      - Returns the unrealised capital in your account in a given bar
  6. total_balance(bar)
      - Unrealised + Realised
  7. store_results()
          - Stores the results in a dataframe
  8. performance()
      - Calculates the performance ratios that is used to evaluate performance of strategy
      - Ratios used:
          1. Sharpe Ratio
          2, ROI (Return on investment)
          3. Drawdown
  9. performance_plots()
      - Visualisations of returns of stocks VS strategy
      - Plot 1: Movement of close prices of all stocks over stated period
      - Plot 2: Histogram of strategy returns
      - Plot 3: Strategy vs Instrument returns
      - Plot 4: Drawdown
      - Plot 5: Equity Curve
  10. buy_order()
      - Conducts a buy order upon fulfilling buy conditions of strategy
  11. sell_order()
      - Conducts a sell order upon fulfilling sell conditions of strategy
  12. last_trade()
      - Checks the last trade conducted
  """
  def __init__(self, symbol, start, end, interval, capital, transcation_cost, verbose = True):

    #new dicts as now there will be multiple stocks. Each stock will have its own position/quantity etc
    self.all_data ={}
    self.position = {}
    self.quantity = {}
    self.stoptrade = {}
    self.boughtprice = {}

    for stock in symbol:
      self.quantity['{}'.format(stock)] = 0 #can also do at run strategy, but its different design

    self.symbol = symbol
    self.start = start
    self.end = end
    self.interval = interval
    self.initial_capital = capital # this is the initial capital you want to trade with
    self.capital = capital # this capital will change depending on trades
    self.transaction_cost = transcation_cost # the transaction cost for trading
    self.trades = 0 # Number of trades
    self.verbose = verbose # if you want to see detailed output (logs)
    self.stored_data = pd.DataFrame(columns = ['trade', 'date', 'position', 'price', 'symbol', 'quantity', 'capital']) # to store trade details

    self.prepare_data() # prepares the data

  def prepare_data(self):
    # since we are building a common class for all types of strategy, we will not calcualte the moving averages now.
    # we will calculate the returns though.
    # Since most strategies utilise close prices we are only factoring close price. However, you can alter acoordingly.

    for i in self.symbol:
      stock_data = yf.Ticker(i)
      hist_stock = stock_data.history(start = self.start, end = self.end, interval = self.interval)
      bt_data = pd.DataFrame()
      bt_data["Close_Price"] = hist_stock["Close"]
      bt_data["Return"] = np.log(bt_data["Close_Price"] / bt_data["Close_Price"].shift(1))
      bt_data = bt_data.dropna()
      self.all_data['{}'.format(i)] = bt_data

  def close_graph(self, stock):
    i = str(stock)
    plt.figure(figsize=(15, 5))
    plt.plot(self.all_data[i]["Close_Price"] ,color='black', label='Price', linestyle='dashed')
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.title("Close Prices of {}".format(i))
    plt.legend()
    plt.grid()
    plt.show()

  def return_date_price(self, bar, stock): #now we will also pass the stock as parameter
    # A bar is a unit of data at a given time, depends on the interval you choose, it provides you OHLCV and time info
    # Since we have modeled close prices, we will get the price and date
    i = str(stock)
    date = str(self.all_data[i].index[bar])[:10]  # :10 to only get the date
    price = self.all_data[i].Close_Price.iloc[bar]
    return date, price

  def realised_balance(self, bar):
    #returns the realised capital in your account at a given time period / bar
    date, price = self.return_date_price(bar, self.symbol[0]) # only concerned with date so any stock will do
    print("Date :{} | Realised Balance: {:0.1f}".format(date,self.capital))

  def unrealised_balance(self, bar):
    #returns you the unrealised capital (trades in progress) in your account at a given time period / bar
    ub = 0
    for stock in self.symbol:
      date, price = self.return_date_price(bar, stock)
      ub =  ub + self.quantity['{}'.format(stock)] *price
    #print("Date :{} | Unrealised Balance: {:0.1f}".format(date,ub))

  def total_balance(self, bar):
    #Unrealised plus realised
    ub = 0
    for stock in self.symbol:
      date, price = self.return_date_price(bar, stock)
      ub =  ub + self.quantity['{}'.format(stock)] *price
    tb = ub + self.capital
    #print("Date :{} | Total Balance: {:0.1f}".format(date,tb))
    return tb

  def store_results(self, trade, date, position, price, symbol, quantity, capital):
    trade_detail = pd.DataFrame({'trade':pd.Series(trade), 'date':pd.Series(date), 'position':pd.Series(position), 'price':pd.Series(price), 'symbol':pd.Series(symbol), 'quantity':pd.Series(quantity), 'capital':pd.Series(capital) })
    self.stored_data = pd.concat([self.stored_data, trade_detail])

  def performance_plots(self):# once we have dataframe of daily account balances
    Final_result_data = self.Final_result_data #saving time to write self everywhere
    interval = str(self.interval)
    f,ax = plt.subplots(2,3,figsize=(20,10))

    #plot 1: Close Price os stock
    plt.subplot(2, 3, 1)
    for stock in self.symbol:
      plt.plot(Final_result_data['Returns_{}'.format(stock)].cumsum(), label='{}'.format(stock), linestyle='dashed')
    plt.title("Cum Log returns of included stocks")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    #plot 2: Histogram of dtrstegy returns
    plt.subplot(2, 3, 2)
    plt.hist(Final_result_data['capital_log_returns'], bins = 35, label='Strategy Returns', linestyle='dashed')
    plt.title("Histogram of Strategy returns")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    #plot 3: Strategy vs Instrument
    plt.subplot(2, 3, 3)
    plt.plot(Final_result_data['Total_Stock_Returns_cum_sum'],color='red', label='Equal weighted stock Returns', linestyle='dashed')
    plt.plot(Final_result_data['capital_regular_returns_cumsum'].dropna(), label ='Strategy Returns')
    plt.title("Strategy Vs Stock Cum Return Curve")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    #plot 4: Drawdown
    plt.subplot(2, 3, 4)
    plt.plot(Final_result_data["capital_returns_log_cum_sum"].dropna(), label='Cum log returns')
    plt.plot(Final_result_data["Cum_Max"].dropna(), label =  'Max Drawdown')
    plt.title("Drawdown")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    #plot 5: Equity Curve
    plt.subplot(2, 3,  5)
    plt.plot(Final_result_data["capital"].dropna(), label =  'Equity $')
    plt.title("Equity Curve")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    plt.suptitle("Performance Plots")
    f.delaxes(ax[1,2]) # to delete the last subplot

    plt.show()

  def performance(self):
    # stored data has  [trade, date, position, price, symbol, quantity, capital]
    #B = self.stored_data # this will have many rows ; n(number of instruments) times more
    #C = self.all_data # will need this to aggregate the returns from all stocks
    #B.index = range(B.shape[0]/len(self.symbol))

    #Since the stocks are not stored in arranged manner, we need to arrange them so that we can get the entry price and exit price for each day
    #stored data has many rows, we will aggregate day wise

    df = pd.DataFrame() # creating a new datframe to store aggregated values

    df['capital'] = self.stored_data.groupby('date').agg({'capital': 'mean'})
    # agrregating by mean because the run is done in loop (for all stocks in a list of symbols),
    # so each stock "date" will have the same capital accounting for all stocks
    # you can print and see different metrics..

    # df.index = range(self.stored_data.shape[0]/len(self.symbol)) # shape[0] to get the length of rows of each stock and make an index for dataframe df

    trade_start_date = df['capital'].index[0] # If intraday, then index needs to be adjusted with time, for daily, its fine
    trade_end_date = df['capital'].index[-1] # This will work when interval is 1D

    #Also, since we have multiple stocks, we need to calculate expected returns to comapre against the strategy

    for stock in self.symbol:
      A = (self.all_data[stock][trade_start_date:trade_end_date]['Return'].tolist()) # The returns are already log
      df['Returns_{}'.format(stock)] = A
    c= df.columns #fetch all the columns of dataframe
    df['Total_Stock_Returns'] = (df.loc[:,c[-len(self.symbol):]].sum(1))/len(self.symbol) # Sums up last N columns of df and divide by number of stocks. Equal weightage
    df['Total_Stock_Returns_cum_sum'] = df['Total_Stock_Returns'].cumsum()
    df.index = range(df.shape[0])

    df['capital_after_settlement'] = df['capital'].shift(-1)

    # Since we are are utilising capital, we will calculate returns for SR based on daily capital/equity changes
    #Remember, you can change the returns on investment based on the capital you trade

    df['capital_regular_returns'] = (df['capital_after_settlement']/df['capital'] - 1)
    df['capital_regular_returns_cumsum'] = df['capital_regular_returns'].cumsum()
    df['capital_log_returns'] = np.log(df['capital_after_settlement']/df['capital'])
    df["capital_returns_log_cum_sum"] = df["capital_log_returns"].cumsum()

    # for drawdown calculation
    df["Cum_Max"] = df["capital_returns_log_cum_sum"].cummax()
    df["drawdown"] = df["Cum_Max"] - df["capital_returns_log_cum_sum"]
    # The maximum difference is refered as "Drawdown"

    df.drop(df.index[df['capital_log_returns'] == 0.0], inplace=True) #Dropping the last column, as I already got the values through shift(1)
    df = df.dropna()
    #print(df.head())

    '''
    #arranging the columns for better readeability
    cols =['trade','symbol','position','quantity','date', 'exit_date', 'price','exit_price',\
           'price_log_returns','price_log_returns_cum_sum','capital',\
          'capital_after_settlement', 'capital_regular_returns', 'capital_log_returns',\
           'capital_returns_log_cum_sum','Cum_Max','drawdown']
    B = B[cols]
    B.rename(columns = {'date':'entry_date', 'price':'entry_price'}, inplace = True)

    '''

    #storing the results in Final_result data
    Final_result_data = df
    #Final_result_data.head()

    #Plot the equity curve
    #plt.plot(Final_result_data['exit_date'],Final_result_data['capital_after_settlement'])
    #Final_result_data[['exit_date', 'capital_after_settlement']].dropna().plot(figsize =(15,6), title = "Equity /Capital Plot")

    print("=" *50)
    print("Performance Metrics")

    #Total Balance
    print("The total capital at end of strategy: {:0.1f}".format(self.capital))

    # Annual gross expected returns
    daily_log_mean_ret = Final_result_data['capital_log_returns'].mean()
    annual_log_ret = 252 * daily_log_mean_ret
    annual_regular_return = (np.exp(annual_log_ret) -1)*100
    print("The annual regular returns: {:0.2f} %".format(annual_regular_return))

    # Annual std deviation
    daily_regular_std = (np.exp(Final_result_data['capital_log_returns'])-1).std()
    annual_regular_std =  daily_regular_std * (252 **0.5) *100
    print("The annual regular std deviation: {:0.2f} %".format(annual_regular_std))

    #Annual Sharpe Ratio
    SR = annual_regular_return / annual_regular_std
    print("The annual Sharpe Ratio: {:0.2f}".format(SR))

    #The strategy returns on investment in percent

    #Total_Log_ret = Final_result_data['capital_log_returns'].sum()
    #Total_Log_ret
    #Regular_ROI = (np.exp(Total_Log_ret) - 1) *100
    #print("The returns on investment from {} to {} by this strategy is {:0.2f} %".format(self.start, self.end, Regular_ROI))

    # the above method will also produce same result

    returns = (self.capital - self.initial_capital) /self.initial_capital *100
    print("The returns on investment from {} to {} by this strategy is {:0.1f} %".format(self.start, self.end, returns))

    #Calculate Drawdown
    #Final_result_data[["capital_returns_log_cum_sum", "Cum_Max"]].dropna().plot(figsize =(15,6), title = "Drawdown Plot")

    Drawdown = Final_result_data["drawdown"].max()*100
    print("The maximum drawdown: {:.2f} %".format(Drawdown))

    zero_periods = Final_result_data[Final_result_data['drawdown'] == 0]
    delta_values = (zero_periods.index[1:] - zero_periods.index[:-1])
    print("The maximum drawdown period: {:.2f} days".format(delta_values.max()))

    #Annual Sortino Ratio
    #Calculate the sortino ratio from the Final result data
    #sortino = ...
    #print("The annual Sortino Ratio: {:0.2f}".format(sortino))

    #Calculate any other ratio you want...

    #Total trades
    print("The total trades taken by the strategy: {}".format(self.trades))


    print("=" *50)

    self.Final_result_data = Final_result_data
    #print(self.Final_result_data.head())


  def buy_order(self,bar,stock,quantity=None, dollar =None ):
    date, price = self.return_date_price(bar,stock)
    if quantity == None:
      quantity = int(dollar/price)
    self.capital = self.capital - ((quantity * price)*(1 + self.transaction_cost)) # capital will be lost in buying
    self.quantity['{}'.format(stock)] = self.quantity['{}'.format(stock)] + quantity
    self.trades = self.trades + 1
    self.position['{}'.format(stock)] = 1
    tb = self.total_balance(bar)
    #self.quantity['{}'.format(stock)]*price + self.capital # this is just for one stock, however, I need for all the stocks
    self.store_results(self.trades, date, self.position['{}'.format(stock)], price, stock, quantity, tb)
    self.boughtprice['{}'.format(stock)] = price


    if self.verbose:
      print("Bought {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(quantity,stock, price, quantity * price))
      print('Total quantity for {} is {}'.format(stock,self.quantity['{}'.format(stock)]))
      self.realised_balance(bar)
      #self.unrealised_balance(bar)
      #self.total_balance(bar)


  def sell_order(self,bar,stock, quantity=None, dollar=None ):
    date, price = self.return_date_price(bar, stock)
    if quantity == None:
      quantity = int(dollar/price)
    self.capital = self.capital + ((quantity * price)*(1 - self.transaction_cost)) # capital will be added after selling
    self.quantity['{}'.format(stock)] = self.quantity['{}'.format(stock)] - quantity
    self.trades = self.trades + 1
    self.position['{}'.format(stock)] = -1
    tb = self.total_balance(bar) # to store the total balance due to all the stocks at the end of the day, so that I get 1 value at the EOD
    self.store_results(self.trades, date, self.position['{}'.format(stock)], price, stock, quantity, tb)
    self.boughtprice['{}'.format(stock)] = price


    if self.verbose:
      print("Sold {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(quantity,stock, price, quantity * price))
      print('Total quantity for {} is {}'.format(stock,self.quantity['{}'.format(stock)]))
      self.realised_balance(bar)
      #self.unrealised_balance(bar)
      #self.total_balance(bar)

  def last_trade(self, bar):# thi sneeds to be check for all open positions of all stocks

    for stock in self.symbol:
      date, price = self.return_date_price(bar, stock)
      last_quantity = self.quantity['{}'.format(stock)]
      self.capital = self.capital + last_quantity * price
      self.quantity['{}'.format(stock)] = 0 # as no more quantity now. all will be settled
      self.trades = self.trades +1
      if self.position['{}'.format(stock)] == -1: # if closing out a short position
        last_quantity = - last_quantity # to keep a positve nymber in records. Note, self.quantity is always maintianing a +/- sign.
        self.position['{}'.format(stock)] = 1 # to record we are buying to close out
      else:
        last_quantity =  last_quantity # to keep a positve nymber in records
        self.position['{}'.format(stock)] = -1 # to record we are selling to close out
      #self.position = 0
      tb = self.total_balance(bar)
      self.store_results(self.trades, date, self.position['{}'.format(stock)], price, stock, last_quantity, tb)

      if self.verbose:
        print("Closed open trades for {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(last_quantity,stock, price, last_quantity * price))

In [8]:
class MA_RSI_Strategy(Common_Class):
  """
    A class dictating our strategy conditions and the corresponding actions. This class inherits the Common_Class above.
    Methods in class:
    1. go_long(bar, stock):
        - Actions to conduct when a long position is taken
    2. go_short(bar, stock):
        - Actions to conduct when a short position is taken
    3. run_strategy(STMA_window, LTMA_window, RSI_window, upper_bound, lower_bound, stop_loss, take_profit):
        - Strategy conditions:
            - Long: LTMA > STMA and RSI < lower_bound
            - Short: LTMA < STMA and RSI > upper_bound
  """

  def go_long(self, bar, stock, quantity = None, dollar = None): # either previously in position or fresh position
    if self.position['{}'.format(stock)] == -1:
      self.position['{}'.format(stock)] = 0
      self.buy_order(bar,stock, quantity = -self.quantity['{}'.format(stock)]) #to clear previous short position and therefore negative quantity.(already in position, thats why negative)
      if dollar == 0: # stop loss or take profit condition
        self.stoptrade['{}'.format(stock)] = -1
        return None
    if quantity:
      self.buy_order (bar, stock, quantity = quantity) # to create new fresh order
      self.stoptrade['{}'.format(stock)] = 0
    elif dollar:
      if dollar == 'all':
        dollar = self.capital
      else:
        dollar = self.capital/len(self.symbol)
      self.buy_order(bar, stock, dollar = dollar)
      self.stoptrade['{}'.format(stock)] = 0

  def go_short(self, bar, stock, quantity = None, dollar = None):
    if self.position['{}'.format(stock)] == 1:
      self.position['{}'.format(stock)] = 0
      self.sell_order(bar,stock, quantity = self.quantity['{}'.format(stock)]) #to clear previous long vposition
      if dollar == 0: # stop loss or take profit condition
        self.stoptrade['{}'.format(stock)] = 1
        return None
    if quantity:
      self.sell_order (bar, stock, quantity = quantity) # to create new fresh order
    elif dollar:
      if dollar == 'all':
        dollar = self.capital
      else:
        dollar = self.capital/len(self.symbol)
      self.sell_order(bar, stock, dollar = dollar)
      self.stoptrade['{}'.format(stock)] = 0

  def run_strategy(self, STMA_window, LTMA_window, RSI_window, upper_bound, lower_bound, stop_loss, take_profit): # add the parameters according to the strategy
    self.stop = False # I will use this to make sure NO trading when balance = 0 OR lost all money
    self.trades = 0
    self.capital = self.initial_capital


    #Preparing data with LTMA and STMA so that can check crossovers. Also initialising with 0 quantity and 0 position

    for stock in self.symbol:
      self.quantity['{}'.format(stock)] = 0
      self.position['{}'.format(stock)] = 0

      indicator_1 = SMAIndicator(close = self.all_data['{}'.format(stock)]["Close_Price"], window = STMA_window, fillna= False)
      STMA = indicator_1.sma_indicator()

      indicator_2 = SMAIndicator(close = self.all_data['{}'.format(stock)]["Close_Price"], window = LTMA_window, fillna= False)
      LTMA = indicator_2.sma_indicator()

      self.all_data['{}'.format(stock)]['STMA_{}'.format(stock)] = STMA
      self.all_data['{}'.format(stock)]['LTMA_{}'.format(stock)] = LTMA

      C = RSIIndicator(close = self.all_data['{}'.format(stock)]["Close_Price"], window= RSI_window, fillna = False)
      self.all_data['{}'.format(stock)]['RSI_{}'.format(stock)] = C.rsi()

      #initialise the bought-in price and indicator for take profit/stop loss
      self.stoptrade['{}'.format(stock)] = 0 #1 indicates a previous take profit/stop loss action at long position, -1 indicates a previous action at short positin
      date, price = self.return_date_price(max(LTMA_window, RSI_window), stock) #initialise with first price
      self.boughtprice['{}'.format(stock)] = price

    for bar in range(max(LTMA_window, RSI_window), len(self.all_data['{}'.format(self.symbol[0])])): # from the available data bar to the last data bar
      #Making a func so that no trade if balance goes below 0

      tb = self.total_balance(bar)
      if tb < 0:
        print("negative capital")
        self.last_trade(bar)
        self.stop = True
        break

      for stock in self.symbol:
        date, price = self.return_date_price(bar, stock)


        if self.position['{}'.format(stock)] in [0,-1]: # checking no position or short position
          if self.position['{}'.format(stock)] in [-1]: # check for price change to stop loss/take profit when in position
            # buy to stop loss or take profit
            if price/self.boughtprice['{}'.format(stock)] - 1 >= stop_loss or price/self.boughtprice['{}'.format(stock)] - 1 <= -take_profit:
              self.go_long(bar,stock, dollar = 0)
          # go long if condition is satisfied and the previous trade is not take profit/stop loss at long position
          if (self.all_data['{}'.format(stock)]['STMA_{}'.format(stock)].iloc[bar] > self.all_data['{}'.format(stock)]['LTMA_{}'.format(stock)].iloc[bar]\
          or self.all_data['{}'.format(stock)]['RSI_{}'.format(stock)].iloc[bar] < lower_bound) and self.stoptrade['{}'.format(stock)] != 1:
             self.go_long(bar,stock, dollar = self.capital/len(self.symbol)) # go with equal money # Make sure to enter less than initila capital
             #print("--------")
             continue # as I am storing data for SR calculations. Dont wnat duplication

        if self.position['{}'.format(stock)] in [0,1]: # checking no position or long position
          if self.position['{}'.format(stock)] in [1]: # check for price change to stop loss/take profit when in position
            # sell to stop loss or take profit
            if price/self.boughtprice['{}'.format(stock)] - 1 <= -stop_loss or price/self.boughtprice['{}'.format(stock)] - 1 >= take_profit:
              self.go_short(bar,stock, dollar = 0)
          if (self.all_data['{}'.format(stock)]['STMA_{}'.format(stock)].iloc[bar] < self.all_data['{}'.format(stock)]['LTMA_{}'.format(stock)].iloc[bar]\
            or self.all_data['{}'.format(stock)]['RSI_{}'.format(stock)].iloc[bar] > upper_bound) and self.stoptrade['{}'.format(stock)] != -1:
            self.go_short(bar,stock, dollar =self.capital/len(self.symbol)) # go with equal money
            #print("--------")
            continue

        #-----Storing all the values for calculating Sharpe-----
        store_quantity = abs(self.quantity['{}'.format(stock)]) # so that I want just the positive numbers in my performance table. I will calculate +/- based on position sign
        tb = self.total_balance(bar)
        self.store_results(0, date, self.position['{}'.format(stock)], price, stock, store_quantity, tb)

    #print("=" *50)
    #print("End of last bar") Now the bar ends, bar is at the last bar value.
    #print("=" *50)
    if self.stop == False: # to make sure not to run this func 2 times when early exit due to negative balance
      self.last_trade(bar)  # amke sure the crossover happens, give more time


In [9]:
top_20_models = []

first_10 = df.iloc[:10].list
second_10 = df.iloc[10:20].list
third_10 = df.iloc[20:30].list
fourth_10 = df.iloc[30:40].list
fifth_10 = df.iloc[40:50].list
sixth_10 = df.iloc[50:60].list
seventh_10 = df.iloc[60:70].list
eighth_10 = df.iloc[70:80].list
nineth_10 = df.iloc[80:90].list
tenth_10 = df.iloc[90:100].list
eleventh_10 = df.iloc[100:110].list
twelfth_10 = df.iloc[110:120].list
thirteenth_10 = df.iloc[120:130].list
fourtheeth_10 = df.iloc[130:140].list
fifteenth_10 = df.iloc[140:150].list
sixteenth_10 = df.iloc[150:160].list
seventeeth_10 = df.iloc[160:170].list
eighteenth_10 = df.iloc[170:180].list
nineteenth_10 = df.iloc[180:190].list
twentieth_10 = df.iloc[190:].list

## 1st Run

In [10]:
model_1 =[]

for i in range(len(first_10)):
    stocks = eval(first_10[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_1.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [11]:
model_result = np.array(model_1)
sorted_index = np.argsort(model_1)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'first_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 7 is 26.09440938472748
ROI for model 5 is 16.796030192947388
ROI for model 8 is 11.622638096618653
ROI for model 1 is 8.710037646102906
ROI for model 2 is 6.161141872406006
ROI for model 4 is 6.120692855453491
ROI for model 9 is 3.4774199003219604
ROI for model 0 is 1.889714962387085
ROI for model 3 is -3.8704855739593507
ROI for model 6 is -4.107283016204834


## 2nd Run

In [12]:
model_2 =[]

for i in range(len(second_10)):
    stocks = eval(second_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_2.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
model_result = np.array(model_2)
sorted_index = np.argsort(model_2)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'second_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 5 is 26.150331554031368
ROI for model 7 is 19.20420892715454
ROI for model 0 is 12.654123646354677
ROI for model 1 is 11.665440309143065
ROI for model 3 is 2.27040598526001
ROI for model 6 is 1.9416319672584534
ROI for model 4 is 1.8763060381889345
ROI for model 8 is -0.06311938819885253
ROI for model 9 is -8.849165545654298
ROI for model 2 is -9.019529880905152


## 3rd Run

In [14]:
model_3 =[]

for i in range(len(third_10)):
    stocks = eval(third_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_3.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [15]:
model_result = np.array(model_3)
sorted_index = np.argsort(model_3)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'third_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 5 is 18.456195805191992
ROI for model 8 is 17.874887478637696
ROI for model 6 is 16.556614415740967
ROI for model 4 is 13.518664233970643
ROI for model 7 is 11.417609497070313
ROI for model 1 is 8.480274625587464
ROI for model 3 is 7.901963983535766
ROI for model 9 is 6.043852766799927
ROI for model 2 is -3.532195750904083
ROI for model 0 is -4.307699406242371


## Run 4

In [16]:
model_4 =[]

for i in range(len(fourth_10)):
    stocks = eval(fourth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_4.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [17]:
model_result = np.array(model_4)
sorted_index = np.argsort(model_4)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'fourth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 4 is 22.251592721939087
ROI for model 1 is 14.006778816604616
ROI for model 8 is 7.881990104675293
ROI for model 9 is 2.256102010536194
ROI for model 5 is -1.5386429355621338
ROI for model 7 is -2.6344090084075926
ROI for model 6 is -7.527787420845032
ROI for model 2 is -11.264437430810927
ROI for model 3 is -11.896712312030793
ROI for model 0 is -12.483060961532592


## Run 5

In [18]:
model_5 =[]

for i in range(len(fifth_10)):
    stocks = eval(fifth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_5.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
model_result = np.array(model_5)
sorted_index = np.argsort(model_5)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'fifth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 0 is 38.27141823730469
ROI for model 1 is 26.761266413116452
ROI for model 4 is 20.74383105516434
ROI for model 5 is 17.46485464916229
ROI for model 9 is 16.44072446899414
ROI for model 7 is 13.807448970985412
ROI for model 6 is 11.265550575256347
ROI for model 2 is 4.885991831588745
ROI for model 8 is 3.90823657913208
ROI for model 3 is -8.047136543273927


## Run 6

In [20]:
model_6 =[]

for i in range(len(sixth_10)):
    stocks = eval(sixth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_6.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [21]:
model_result = np.array(model_6)
sorted_index = np.argsort(model_6)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'sixth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 6 is 22.42636255950928
ROI for model 8 is 17.10896625747681
ROI for model 0 is 13.284074568176269
ROI for model 4 is 6.895779276657104
ROI for model 9 is 3.2841568662643437
ROI for model 5 is 1.3600329971313476
ROI for model 3 is 0.9286840511322022
ROI for model 2 is -0.35220671920776364
ROI for model 7 is -1.6800938308715823
ROI for model 1 is -7.181027261734009


## Run 7

In [22]:
model_7 =[]

for i in range(len(seventh_10)):
    stocks = eval(seventh_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_7.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [23]:
model_result = np.array(model_7)
sorted_index = np.argsort(model_7)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'seventh_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 0 is 22.411887175750735
ROI for model 4 is 20.191124332237244
ROI for model 5 is 19.183666929244993
ROI for model 9 is 13.685568587112426
ROI for model 3 is 8.703866053962708
ROI for model 6 is 8.068401177597046
ROI for model 1 is 7.4229248302459725
ROI for model 8 is 5.743841104888916
ROI for model 7 is 0.8864111413955689
ROI for model 2 is -1.1383174016952515


## Run 8

In [24]:
model_8 =[]

for i in range(len(eighth_10)):
    stocks = eval(eighth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_8.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [25]:
model_result = np.array(model_8)
sorted_index = np.argsort(model_8)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'eighth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 0 is 30.874065236663817
ROI for model 7 is 25.458664429473878
ROI for model 4 is 21.05199208869934
ROI for model 2 is 18.809076946258546
ROI for model 1 is 13.521918530750273
ROI for model 3 is 13.32197758102417
ROI for model 9 is 11.538408687973023
ROI for model 8 is 7.65205228395462
ROI for model 6 is 6.494231641960144
ROI for model 5 is 2.104816075134277


## Run 9

In [26]:
model_9 =[]

for i in range(len(nineth_10)):
    stocks = eval(nineth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_9.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [27]:
model_result = np.array(model_9)
sorted_index = np.argsort(model_9)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'nineth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 9 is 22.52246762866974
ROI for model 0 is 19.950617914199828
ROI for model 2 is 19.419985147094724
ROI for model 7 is 13.394695557785033
ROI for model 3 is 7.065889306831361
ROI for model 8 is 4.963689834022523
ROI for model 4 is 3.878514332771301
ROI for model 6 is 2.5023086290359497
ROI for model 5 is -4.331215912246704
ROI for model 1 is -9.321377237510681


## Run 10

In [28]:
model_10 =[]

for i in range(len(tenth_10)):
    stocks = eval(tenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_10.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [29]:
model_result = np.array(model_10)
sorted_index = np.argsort(model_10)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'tenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 0 is 34.24010028190613
ROI for model 6 is 20.114829022407534
ROI for model 4 is 11.447666276550294
ROI for model 5 is 8.942350095176698
ROI for model 1 is 8.453542738056182
ROI for model 9 is 7.339345030593872
ROI for model 8 is 6.334135780334472
ROI for model 2 is 2.692531516647339
ROI for model 7 is 2.4680029998779296
ROI for model 3 is -1.1171294982910156


## Run 11

In [30]:
model_11 =[]

for i in range(len(eleventh_10)):
    stocks = eval(eleventh_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_11.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [31]:
model_result = np.array(model_11)
sorted_index = np.argsort(model_11)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'eleventh_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 5 is 19.272589960861204
ROI for model 0 is 13.289131748580932
ROI for model 6 is 3.048604653549194
ROI for model 7 is 1.7803676181793213
ROI for model 3 is 1.7482696548461913
ROI for model 9 is -2.2307868927955625
ROI for model 8 is -2.9479489671707153
ROI for model 4 is -4.310442289733887
ROI for model 1 is -18.156814273643494
ROI for model 2 is -19.644287971115112


## Run 12

In [32]:
model_12 =[]

for i in range(len(twelfth_10)):
    stocks = eval(twelfth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_12.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [33]:
model_result = np.array(model_12)
sorted_index = np.argsort(model_12)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'twelfth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 0 is 28.810032246208188
ROI for model 6 is 27.589611846923827
ROI for model 7 is 21.748061278915408
ROI for model 4 is 19.31887387981415
ROI for model 9 is 11.769561239624023
ROI for model 8 is 10.82914281616211
ROI for model 2 is 0.41887861328125
ROI for model 3 is -2.477601531791687
ROI for model 1 is -8.92750684776306
ROI for model 5 is -13.305881155014038


## Run 13

In [34]:
model_13 =[]

for i in range(len(thirteenth_10)):
    stocks = eval(thirteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_13.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [35]:
model_result = np.array(model_13)
sorted_index = np.argsort(model_13)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'thirteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 4 is 21.78183211736679
ROI for model 6 is 21.084169732284547
ROI for model 2 is 20.31699546813965
ROI for model 5 is 6.0131688320159915
ROI for model 0 is 5.543069004440308
ROI for model 3 is 4.767757571411132
ROI for model 7 is 2.872040099334717
ROI for model 8 is -2.792818726348877
ROI for model 1 is -3.6690608314514157
ROI for model 9 is -14.374344139289855


## Run 14

In [36]:
model_14 =[]

for i in range(len(fourtheeth_10)):
    stocks = eval(fourtheeth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_14.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [37]:
model_result = np.array(model_14)
sorted_index = np.argsort(model_14)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'fourtheeth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 9 is 24.17433119087219
ROI for model 0 is 23.98181902427673
ROI for model 5 is 22.323698763847354
ROI for model 1 is 15.927425414657591
ROI for model 7 is 8.166228677749633
ROI for model 8 is 7.979506813621522
ROI for model 6 is 7.897243217658996
ROI for model 2 is 7.38255191745758
ROI for model 4 is 6.175359225273133
ROI for model 3 is 4.605029457378388


## Run 15

In [38]:
model_15 =[]

for i in range(len(fifteenth_10)):
    stocks = eval(fifteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_15.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [39]:
model_result = np.array(model_15)
sorted_index = np.argsort(model_15)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'fifteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 2 is 34.78823162841797
ROI for model 1 is 27.576831098651887
ROI for model 0 is 14.327906816482544
ROI for model 5 is 14.069986821174622
ROI for model 7 is 9.208847203254699
ROI for model 8 is 1.37054597530365
ROI for model 4 is -0.005713579559326171
ROI for model 3 is -1.20916491355896
ROI for model 9 is -3.70329441986084
ROI for model 6 is -11.194743317794801


## Run 16

In [40]:
model_16 =[]

for i in range(len(sixteenth_10)):
    stocks = eval(sixteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_16.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [41]:
model_result = np.array(model_16)
sorted_index = np.argsort(model_16)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'sixteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 1 is 18.40511415271759
ROI for model 5 is 15.722382590293885
ROI for model 3 is 11.744730858039857
ROI for model 2 is 6.826950384521484
ROI for model 0 is 4.530825149154663
ROI for model 8 is 2.7963024879455567
ROI for model 9 is 2.5791091054916384
ROI for model 6 is 0.4273230952262878
ROI for model 4 is -2.723403624343872
ROI for model 7 is -14.167511545753477


## Run 17

In [42]:
model_17 =[]

for i in range(len(seventeeth_10)):
    stocks = eval(seventeeth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_17.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [43]:
model_result = np.array(model_17)
sorted_index = np.argsort(model_17)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'seventeeth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 5 is 44.620454566955566
ROI for model 9 is 32.3375505897522
ROI for model 0 is 18.053809712028503
ROI for model 6 is 15.161145708465575
ROI for model 1 is 13.522880668449403
ROI for model 3 is 7.932233429908752
ROI for model 7 is 0.6255115734100342
ROI for model 4 is -1.2103309469223023
ROI for model 8 is -1.7081648778915406
ROI for model 2 is -2.7182885303497315


## Run 18

In [44]:
model_18 =[]

for i in range(len(eighteenth_10)):
    stocks = eval(eighteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_18.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [45]:
model_result = np.array(model_18)
sorted_index = np.argsort(model_18)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'eighteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 1 is 36.529609605026245
ROI for model 4 is 18.371820441246033
ROI for model 7 is 15.268479145812988
ROI for model 9 is 11.748125077056885
ROI for model 0 is 10.464277146148682
ROI for model 6 is 9.55552712097168
ROI for model 3 is 6.438439234161376
ROI for model 2 is -4.361182493782043
ROI for model 5 is -8.531863394165038
ROI for model 8 is -12.830478278923035


## Run 19

In [46]:
model_19 =[]

for i in range(len(nineteenth_10)):
    stocks = eval(nineteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_19.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [47]:
model_result = np.array(model_19)
sorted_index = np.argsort(model_19)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'nineteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 1 is 31.083984688186643
ROI for model 8 is 20.215858813095092
ROI for model 9 is 17.4992498008728
ROI for model 5 is 10.326886313438417
ROI for model 4 is 5.42942438621521
ROI for model 7 is 4.856658331298829
ROI for model 2 is 3.0183581329345706
ROI for model 0 is 2.421710264968872
ROI for model 6 is 0.14266370697021485
ROI for model 3 is -0.3462382801055908


## 20th Run

In [48]:
model_20 =[]

for i in range(len(twentieth_10)):
    stocks = eval(twentieth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_20.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [49]:
model_result = np.array(model_20)
sorted_index = np.argsort(model_20)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'twentieth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 8 is 21.62563275680542
ROI for model 6 is 17.45146227416992
ROI for model 5 is 15.683810736846922
ROI for model 2 is 12.745555879592896
ROI for model 0 is 4.8504779740810395
ROI for model 1 is 1.3166862670898438
ROI for model 9 is -0.7033651279449463
ROI for model 7 is -1.6621121906280518
ROI for model 4 is -3.9358455566406247
ROI for model 3 is -6.950808643341065


In [50]:
top_20_models

[{'first_10': {7, 26.09440938472748}},
 {'second_10': {5, 26.150331554031368}},
 {'third_10': {5, 18.456195805191992}},
 {'fourth_10': {4, 22.251592721939087}},
 {'fifth_10': {0, 38.27141823730469}},
 {'sixth_10': {6, 22.42636255950928}},
 {'seventh_10': {0, 22.411887175750735}},
 {'eighth_10': {0, 30.874065236663817}},
 {'nineth_10': {9, 22.52246762866974}},
 {'tenth_10': {0, 34.24010028190613}},
 {'eleventh_10': {5, 19.272589960861204}},
 {'twelfth_10': {0, 28.810032246208188}},
 {'thirteenth_10': {4, 21.78183211736679}},
 {'fourtheeth_10': {9, 24.17433119087219}},
 {'fifteenth_10': {2, 34.78823162841797}},
 {'sixteenth_10': {1, 18.40511415271759}},
 {'seventeeth_10': {5, 44.620454566955566}},
 {'eighteenth_10': {1, 36.529609605026245}},
 {'nineteenth_10': {1, 31.083984688186643}},
 {'twentieth_10': {8, 21.62563275680542}}]